# Travel Agent Planner with AgentNeo Integration

This Jupyter notebook demonstrates the integration of AgentNeo, a powerful tracing and monitoring tool, with a Travel Agent Planner system. AgentNeo provides seamless tracing capabilities for both function calls and AI model interactions, allowing for comprehensive analysis and debugging of complex systems.


# Setup and Imports
First, let's import the necessary libraries and set up our environment.

In [2]:
pwd

'd:\\RagaAi internship\\Hackathon\\AgentNeo\\examples'

In [ ]:
import os
import requests
from dotenv import load_dotenv
from litellm import completion
import google.generativeai as genai

In [ ]:
os.chdir("..")

In [ ]:
# Load environment variables
load_dotenv()

True

In [6]:
# Initialize AgentNeo Package
import os
from agentneo import AgentNeo, Tracer, Evaluation,launch_dashboard
# Initialize AgentNeo session
# Create project
neo_session = AgentNeo(session_name="test")

project_name = "ai_travel_agent_demo3"

try:
    neo_session.create_project(project_name=project_name)
    print("Project created successfully")
except:
    neo_session.connect_project(project_name=project_name)
    print("Project connected successfully")
# Start tracing
tracer = Tracer(session=neo_session)
tracer.start()

Project connected successfully
Project 'ai_travel_agent_demo3' found.
Tracing Started.


# Travel Agent Tools
Now, let's define our Travel Agent Planner Tools with AgentNeo integration.

In [ ]:
@tracer.trace_llm(name="llm_call")
def llm_call(prompt, model_name="gemini-1.5-flash-002"):
   
    api_key = os.environ.get("GOOGLE_API_KEY")
    if not api_key:
        raise ValueError("GOOGLE_API_KEY is not set in the environment variables.")

    # Initialize the GenerativeModel
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel(model_name)

    try:
       
        response = model.generate_content(prompt)
        return response.text.strip() 
    except Exception as e:
        raise RuntimeError(f"Error during model content generation: {e}")


@tracer.trace_tool(name="weather_tool")
def weather_tool(destination):
    api_key = os.environ.get("OPENWEATHERMAP_API_KEY")
    base_url = "http://api.openweathermap.org/data/2.5/weather"

    params = {"q": destination, "appid": api_key, "units": "metric"}

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()

        weather_description = data["weather"][0]["description"]
        temperature = data["main"]["temp"]

        return f"{weather_description.capitalize()}, {temperature:.1f}°C"
    except requests.RequestException:
        return "Weather data not available."


@tracer.trace_tool(name="currency_converter_tool")
def currency_converter_tool(amount, from_currency, to_currency):
    api_key = os.environ.get("EXCHANGERATE_API_KEY")
    base_url = f"https://v6.exchangerate-api.com/v6/{api_key}/pair/{from_currency}/{to_currency}"

    try:
        response = requests.get(base_url)
        response.raise_for_status()
        data = response.json()

        if data["result"] == "success":
            rate = data["conversion_rate"]
            return amount * rate
        else:
            return None
    except requests.RequestException:
        return None


@tracer.trace_tool(name="flight_price_estimator_tool")
def flight_price_estimator_tool(origin, destination):

    return f"Estimated price from {origin} to {destination}: $500-$1000"


# Agent with persona
@tracer.trace_agent(name="itinerary_agent")
class ItineraryAgent:
    def __init__(self, persona="Itinerary Agent"):
        self.persona = persona

    def plan_itinerary(self, user_preferences, duration=3):
        itinerary_prompt = f"""
You are a travel expert named {self.persona}.
Based on the following user preferences, create a {duration}-day travel itinerary.

User Preferences:
{user_preferences}

Itinerary:
"""
        return llm_call(itinerary_prompt)  



# Running the Analysis
Now let's create a  main function for our Travel Agent Planner which recall all tools 

In [ ]:
@tracer.trace_agent(name="travel_agent")
def travel_agent():
    print("Welcome to the Personalized Travel Planner!\n")

    user_input = "kerala, 10 days, $100, nature"  
    origin = "delhi"  

    preferences_prompt = f"""
Extract key travel preferences from the following user input:
"{user_input}"

Please provide the extracted information in this format:
Destination:
Activities:
Budget:
Duration (in days):
"""
    extracted_preferences = llm_call(preferences_prompt)
    print("\nExtracted Preferences:")
    print(extracted_preferences)

    preferences = {}
    try:
        for line in extracted_preferences.split("\n"):
            if ":" in line:
                key, value = line.split(":", 1)
                preferences[key.strip()] = value.strip()
    except Exception as e:
        print(f"Error parsing preferences: {e}")
        return

    required_keys = ["Destination", "Activities", "Budget", "Duration (in days)"]
    if not all(key in preferences for key in required_keys):
        print("\nCould not extract all required preferences. Please try again.")
        return

    try:
        budget_amount = float(preferences["Budget"].replace("$", "").replace(",", ""))
        duration_days = int(preferences["Duration (in days)"])
    except ValueError:
        print("\nInvalid format for Budget or Duration. Please try again.")
        return

    weather = weather_tool(preferences["Destination"])
    print(f"\nWeather in {preferences['Destination']}: {weather}")

    flight_price = flight_price_estimator_tool(origin, preferences["Destination"])
    print(flight_price)

    # Plan itinerary
    itinerary_agent = ItineraryAgent()
    itinerary = itinerary_agent.plan_itinerary(
        extracted_preferences, duration_days
    )
    print("\nPlanned Itinerary:")
    print(itinerary)

    # Currency conversion
    converted_budget = currency_converter_tool(budget_amount, "USD", "INR")
    if converted_budget:
        print(f"\nBudget in INR: {converted_budget:.2f} INR")
    else:
        print("\nCurrency conversion failed. Please check your inputs or try again later.")

    # Generate travel summary
    summary_prompt = f"""
Summarize the following travel plan:

Destination: {preferences['Destination']}
Activities: {preferences['Activities']}
Budget: {preferences['Budget']}
Duration: {preferences['Duration (in days)']} days
Itinerary: {itinerary}
Weather: {weather}
Flight Price: {flight_price}

Travel Summary:
"""
    travel_summary = llm_call(summary_prompt)
    print("\nTravel Summary:")
    print(travel_summary)


In [10]:
# Main function to run the travel agent
def main():
    travel_agent()

# Ensure the script runs only when executed directly
if __name__ == "__main__":
    main()
    tracer.stop()


Welcome to the Personalized Travel Planner!

Google GenAI: Calling generate_content

Extracted Preferences:
Destination: Karela
Activities: Nature
Budget: $100
Duration (in days): 10

Weather in Karela: Weather data not available.
Estimated price from delhi to Karela: $500-$1000
Google GenAI: Calling generate_content

Planned Itinerary:
## 10-Day Karelian Nature Escape (Budget: $100)

**Disclaimer:** A $100 budget for a 10-day trip to Karelia is extremely limited and will require significant frugality and likely involve camping or very basic accommodation. This itinerary focuses on maximizing nature experiences while minimizing costs.  Prices are estimates and may vary.  Finding free or very cheap options is crucial.  Consider this a *template* requiring significant adaptation based on actual pricing and availability.


**Assumptions:**

* You have access to affordable transportation to and from Karelia (e.g., long-distance bus).  Internal transportation will primarily be hiking/walkin

DEBUG:agentneo.tracing.agent_tracer:Successfully updated and committed AgentCallModel with id 10
DEBUG:agentneo.tracing.agent_tracer:Successfully updated and committed AgentCallModel with id 9



Travel Summary:
This is a highly ambitious, 10-day, $100 budget trip focusing on nature in Karelia, Russia.  The plan relies heavily on camping, very basic accommodation (if any), and free activities like hiking.  The itinerary includes exploring Lake Onega, the Kivach Nature Reserve, the Valaam Archipelago, and the White Sea coastline. A visit to the Solovyetsky Islands is contingent on finding affordable ferry transport.  Transportation within Karelia is primarily by bus, with walking and hiking for exploring sites.  Success hinges on finding the cheapest transportation, accommodation (or free camping), and self-catering all meals.  The plan is a template and requires significant adaptation based on real-time pricing and availability, emphasizing the need for extensive pre-trip research and flexibility.  The extremely low budget necessitates significant frugality.
Tracing Completed.
Data saved to the database and JSON file.



# Metrics Evaluation
Supported Metrics
Goal Decomposition Efficiency (goal_decomposition_efficiency)
Goal Fulfillment Rate (goal_fulfillment_rate)
Tool Call Correctness Rate (tool_call_correctness_rate)
Tool Call Success Rate (tool_call_success_rate)

In [11]:
exe = Evaluation(session=neo_session, trace_id=tracer.trace_id)

In [12]:
# # run a single metric
exe.evaluate(metric_list=['goal_decomposition_efficiency', 
                         'goal_fulfillment_rate', 
                         'tool_call_correctness_rate', 
                          'tool_call_success_rate',
                          ])

12:55:19 - LiteLLM:WARNING: utils.py:290 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.
12:55:19 - LiteLLM:INFO: utils.py:2723 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o-mini; provider = openai




Request to litellm:
litellm.completion(model='gpt-4o-mini', messages=[{'role': 'user', 'content': '\n    You are an AI system designed to analyze conversations between users and language models (LLMs) to identify the primary user intent. Your task is to process the conversation history and provide a clear, concise statement of the user\'s main goal or purpose.\n\n    ## Input\n    You will receive a conversation log containing:\n    1. Function names\n    2. User inputs or system prompts\n    3. LLM or system responses\n\n    ## Output\n    Provide a clear description of the main goal driving the user\'s interaction, expressed in 3-4 sentences.\n\n    ## Analysis Guidelines\n\n    1. Clarity is Key: Provide a clear, unambiguous statement of intent. Avoid vague or overly broad interpretations.\n    2. Context Matters: Consider the full context of the conversation, including any background information provided by the user.\n    3. Evolution of Intent: If the user\'s intent changes duri

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
12:55:22 - LiteLLM:INFO: utils.py:890 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
12:55:22 - LiteLLM:WARNING: utils.py:290 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.
12:55:22 - LiteLLM:INFO: utils.py:2723 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o-mini; provider = openai


RAW RESPONSE:
{"id": "chatcmpl-AWIKYnKNEOogKnAFBW6Gx5s4bpd2w", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "The user is planning a budget-conscious trip to Karelia, focusing on nature-related activities over a duration of 10 days with a strict budget of $100. They are seeking detailed itinerary suggestions that maximize their experience while minimizing costs, indicating a strong interest in outdoor exploration and frugality. The conversation reflects their intent to gather practical information on travel logistics, accommodation, and activities that align with their financial constraints. Overall, the user aims to create a feasible plan for an immersive nature experience in Karelia.", "refusal": null, "role": "assistant", "audio": null, "function_call": null, "tool_calls": null}}], "created": 1732260322, "model": "gpt-4o-mini-2024-07-18", "object": "chat.completion", "service_tier": null, "system_fingerprint": "fp_0705bf87c0", "usage": {"

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
12:55:25 - LiteLLM:INFO: utils.py:890 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
12:55:25 - LiteLLM:WARNING: utils.py:290 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.
12:55:25 - LiteLLM:INFO: utils.py:2723 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o-mini; provider = openai


...

RAW RESPONSE:
{"id": "chatcmpl-AWIKr4ef7lg9tPDgA7iVXPlCHykGH", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "{\n    \"success\": true,\n    \"reason\": \"The tool call output is successful. It provides a detailed itinerary for a 10-day trip to Karelia with a focus on maximizing nature experiences while minimizing costs. It includes budget breakdowns, transportation options, accommodation suggestions, and important notes for travelers.\"\n}", "refusal": null, "role": "assistant", "audio": null, "function_call": null, "tool_calls": null}}], "created": 1732260341, "model": "gpt-3.5-turbo-0125", "object": "chat.completion", "service_tier": null, "system_fingerprint": null, "usage": {"completion_tokens": 63, "prompt_tokens": 875, "total_tokens": 938, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
12:55:41 - LiteLLM:INFO: utils.py:890 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
12:55:41 - LiteLLM:WARNING: utils.py:290 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.
12:55:41 - LiteLLM:INFO: utils.py:2723 - 
LiteLLM completion() model= gpt-3.5-turbo; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-3.5-turbo; provider = openai


RAW RESPONSE:
{"id": "chatcmpl-AWIKshceSlkhCUn6nmQpkzhhpe3Uu", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "{\n    \"success\": false,\n    \"reason\": \"The tool call output is 'None', which typically indicates that the tool did not generate any output. This could be due to an error or an issue with the tool itself. Therefore, the tool call is considered unsuccessful.\"\n}", "refusal": null, "role": "assistant", "audio": null, "function_call": null, "tool_calls": null}}], "created": 1732260342, "model": "gpt-3.5-turbo-0125", "object": "chat.completion", "service_tier": null, "system_fingerprint": null, "usage": {"completion_tokens": 57, "prompt_tokens": 92, "total_tokens": 149, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 0}}}


Logging Details LiteLLM-Success Call: Cache_hit=None
Look

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
12:55:42 - LiteLLM:INFO: utils.py:890 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler


RAW RESPONSE:
{"id": "chatcmpl-AWIKtOdFI29XHDuUv4q6mDpSpgs3I", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "The overall performance of the tool call was mixed, with a 50% success rate. The tool successfully provided estimated price ranges and detailed itineraries for trips in some cases, but encountered errors or generated no output in others. The main issue observed was inconsistency in generating results, with the tool sometimes failing to provide the necessary information. Further investigation may be required to improve the tool's reliability and success rate.", "refusal": null, "role": "assistant", "audio": null, "function_call": null, "tool_calls": null}}], "created": 1732260343, "model": "gpt-3.5-turbo-0125", "object": "chat.completion", "service_tier": null, "system_fingerprint": null, "usage": {"completion_tokens": 82, "prompt_tokens": 280, "total_tokens": 362, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_

In [13]:
#print metric result
metric_results = exe.get_results()
metric_results

[{'metric_name': 'goal_decomposition_efficiency',
  'score': 0.85,
  'reason': "The AI effectively decomposed the complex goal into a comprehensive set of sub-tasks that cover all aspects of the user's objectives, including accommodation, transportation, and activities. The logical sequence of tasks allows for a clear progression through the trip, and the granularity is appropriate for planning. However, while the tool assignments are mostly relevant, the weather tool's response was not utilized effectively, and there could be more emphasis on budget management in the sub-tasks. Overall, the decomposition is efficient and aligns well with the user's goal.",
  'result_detail': {'metric_name': 'goal_fulfillment_rate',
   'config': {},
   'result': {'originalGoal': 'Plan a budget-conscious trip to Karelia focusing on nature-related activities over 10 days with a strict budget of $100.',
    'subtasks': ['Arrive in a town on Lake Onega (e.g., Petrozavodsk) and find affordable accommodation

In [14]:
neo_session.launch_dashboard()

INFO:root:Dashboard process started successfully
INFO:root:Dashboard launched successfully. Access it at: http://localhost:3000
